In [1]:
import logging
import sys
import os

import openai
from dotenv import load_dotenv
load_dotenv()

from llama_index import VectorStoreIndex, SimpleDirectoryReader
from llama_index.storage.storage_context import StorageContext
from llama_index.node_parser import SentenceWindowNodeParser
from llama_index.vector_stores import ElasticsearchStore


logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


os.environ['OPENAI_API_KEY'] = os.environ['OPENAI_API_KEY']
openai.api_key = os.getenv('OPENAI_API_KEY')


class ChatEngine:

    def __init__(self, documents_path="../../data/", new_indexing=False):
        self.vector_store = ElasticsearchStore(
                                es_url="http://localhost:9200",
                                index_name="law_index",
                            )
        self.node_parser = SentenceWindowNodeParser.from_defaults(
            window_size=3,
            window_metadata_key="window",
            original_text_metadata_key="original_text",
        )
        self.documents = SimpleDirectoryReader(documents_path).load_data()
        self.sentence_nodes = self.node_parser.get_nodes_from_documents(self.documents)
        self.storage_context = StorageContext.from_defaults(vector_store=self.vector_store)
        self.index = VectorStoreIndex.from_vector_store(self.vector_store, storage_context=self.storage_context)
        if new_indexing:
            self.index = VectorStoreIndex(
                self.sentence_nodes,
                storage_context=self.storage_context,
            )


    def chat(self, ques="Bạn là ai?"):
        query_engine = self.index.as_query_engine()
        res =query_engine.query(ques)
        return res



In [2]:
r = ChatEngine()
print(r.chat())

INFO:elastic_transport.transport:GET http://localhost:9200/ [status:200 duration:0.022s]
GET http://localhost:9200/ [status:200 duration:0.022s]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:elastic_transport.transport:POST http://localhost:9200/law_index/_search [status:200 duration:0.047s]
POST http://localhost:9200/law_index/_search [status:200 duration:0.047s]
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
I am someone who is closely related to the party being investigated or the complaining party, has rights or obligations related to the competition case, and has clear grounds to believe they are not impartial when performing their duties.
